# Geometries

This lesson brings you a bit back to highschool math: (Euclidian) Geometry, 
hopefully with more fun as within (geo)spatial IT we usually deal with
real-world artifacts (roads, lakes, forests etc).

From [Wikipedia](https://en.wikipedia.org/wiki/Geometry): 

	Geometry (from the Ancient Greek: γεωμετρία; geo- "earth", -metron "measurement") 
	is a branch of mathematics concerned with questions of shape, size, relative 
	position of figures, and the properties of space.

Within the geospatial domain we mainly deal with **Geometries** (at least for Vector data)
where some of the above math applies. Vector data is encoded with coordinates 
mostly X, Y (sometimes also Z for 3D) and used to represent **three basic Geometry types**: 

* Points
* Lines (a.k.a. LineStrings, Curves)
* Polygons (a.k.a. Surfaces)

These are used to represent phenomenons like: 

* city centers, Point of Interests, ... (PoI)  (Points)
* roads, rivers, ... (Lines)
* forests, lakes, countries, ... (Polygons)

To make it a bit more complex: these three basic Geometries are often extended and even
combined to form collections called **"Multi" Geometries**:
 
* a collections of Points is called a *MultiPoint*
* a collection of Lines is called a *MultiLine* (a.k.a. *MultiLineString*) 
* a collection of Polygons is called a  *MultiPolygon* (a.k.a. *MultiSurface*)  

These collections are useful for modeling certain kinds of features. 
A road or river with all its bends is typically a *MultiLine*.
A country like Greece, The Netherlands or Canada is typically 
a *MultiPolygon* (think of: mainland + islands)

We will be mainly working with [Shapely](http://toblerity.org/shapely/manual.html), a Python package for 
set-theoretic analysis and manipulation of, yes, Geometries!
Shapely provides a Spatial Data Model that basically implements
the above Geometry types plus their variants and the (mathematic) manipulations on these.
 
## Background Reading

* https://en.wikipedia.org/wiki/Geometry
* The Shapely User Manual: https://shapely.readthedocs.io/en/stable/manual.html
* https://automating-gis-processes.github.io/CSC/notebooks/L1/geometric-objects.html

## Shapely Basics

Shapely is concerned with Geometries in general, not necessarily Geospatial.
As such we can introduce Shapely basics, using the well-known Euclidian X,Y plane.

### Shapely Points

In [ ]:
from shapely.geometry import Point

point1 = Point(0.0, 0.0)


In [ ]:
point1.area


In [ ]:
point1.length


In [ ]:
point1.wkt


In [ ]:

point2 = Point(3, 4)

point1.distance(point2)


### Shapely LineStrings


In [ ]:
from shapely.geometry import LineString
line = LineString([(0, 0), (3, 4)])


In [ ]:
line.area


In [ ]:
line.length


### Shapely Polygons


In [ ]:
from shapely.geometry import Polygon
polygon = Polygon([(0, 0), (3, 4), (3, 0)])
polygon


In [60]:
# Convert coordinates to list
# polygon.coords does not exist!
list(polygon.exterior.coords)


[(0.0, 0.0), (3.0, 4.0), (3.0, 0.0), (0.0, 0.0)]

In [ ]:
polygon.area


In [ ]:
polygon.length


In [ ]:
# Its x-y bounding box is a (minx, miny, maxx, maxy) tuple.

polygon.bounds


### About Projections and Shapely
In Shapely, the distance is the Euclidean Distance or 
Linear distance (Pythagoras Law!) between two points on a plane and not the 
[Great-circle distance](http://en.wikipedia.org/wiki/Great-circle_distance) between two points on a sphere!
If you are working with data in WGS84 (EPSG:4326), 'lat/lon' (think of GPS coordinates) in degrees,
Shapely's calculations like `length` and `area` will not be what you would expect. 

We have several options (see also [this SE discussion](https://gis.stackexchange.com/questions/80881/what-is-unit-of-shapely-length-attribute)):

* reproject your source data to a 'metric' projection like Web Mercator (EPSG:3857, used for tiles by Google, OSM and others)
* use `pyproj` to apply the proper formulas

Below an example to illustrate:


In [ ]:
from shapely.geometry import Point
import pyproj

point1 = Point(50.67, 4.62)
point2 = Point(51.67, 4.64)

# Shapely Distance in degrees
point1.distance(point2)

In [ ]:
geod = pyproj.Geod(ellps='WGS84')
angle1,angle2,distance = geod.inv(point1.x, point1.y, point2.x, point2.y)

# "Real" Distance in km
distance/1000.0
    

## More About Shapely  

[Shapely](http://toblerity.org/shapely/manual.html) is a Python package for 
set-theoretic analysis and manipulation of planar features using (via Python’s ctypes module) 
functions from the well-known and widely deployed [GEOS library](http://trac.osgeo.org/geos). 
GEOS, a port of the [JTS Topology Suite](http://www.tsusiatsoftware.net/jts/main.html) (JTS), 
is the geometry engine of the PostGIS spatial extension for the PostgreSQL RDBMS.
 
The designs of JTS and GEOS are largely guided by the 
Open Geospatial Consortium‘s (OGC) and ISO 19125 
*Simple Features (Access) Specification* [ref](https://en.wikipedia.org/wiki/Simple_Features). 
Shapely adheres mainly to the same set of standard classes and operations. 
Shapely is thereby deeply rooted in the conventions of the geographic information systems (GIS) world, 
but aspires to be equally useful to programmers working on non-conventional problems.

With Shapely, we can solve tasks like

* What is the area of The Netherlands?
* What is the distance between Amsterdam and Athens?
* How many kilometers is The Donau?
* Do two features overlap?
* How does the common area of two features look like?
* Create a buffer area around the feature?
* ...

### Converting JSON to geometry objects
Fiona will be treated in a next lesson. Here we mainly use Fiona 
to read Vector data (Features) into memory for subsequent Shapely manipulation.

Feature geometry can be accessed using the `geometry` property of each feature, for example
we can open the dataset that contains a (Multi)Polygon for each country in Europe and print
out the geometry of the 10th Feature:

First we import `Shapely` and its functions and then convert the JSON-encoded geometries to Geometry objects
using the `shape` function.

In [ ]:
from shapely.geometry import shape

with fiona.open("../data/europe_countries.3857.json") as eu:
    geom = shape(eu[14]["geometry"])
geom # Jupyter can display geometry data directly

In [ ]:
print(geom.type)

In [ ]:
print(geom.area)

In [ ]:
# In km
print(geom.length/1000)

Let's have a look at some geometry methods. 
Tip: Shapely code is well-documented, you can always use the Python built-in `help()` function.

In [ ]:
help(geom)

For example we can make a buffer of 500 meter around our polygon (making England somewhat bigger):

In [ ]:
buffered_geom = geom.buffer(500)
buffered_geom

In [ ]:
# In km
buffered_geom.length/1000

## Converting the geometry back to JSON format
Once we are finished, we can convert the geometry back to JSON format using `shapely.geometry.mapping` function


In [ ]:
from shapely.geometry import mapping

In [ ]:
# let's create new GeoJSON-encoded vector feature

new_feature = {
    "type": "Feature",
    "properties": {"name": "My buffered feature"},
    "geometry": mapping(geom.buffer(100))
}
new_feature

# Now we could e.g. write the Feature back to file

---
[<- Introduction](01-introduction.ipynb) | [Projections ->](03-projections.ipynb)